# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [3]:
import torch

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [ ]:
m = torch.distributions.normal.Normal(torch.zeros((4096, 4096)), torch.tensor([1.]))
A = m.sample()
B = m.sample()
A, B

(tensor([[ 1.1009,  1.6132,  0.4601,  ..., -1.1506, -0.4024,  0.5553],
         [-0.4577, -1.0078, -1.1603,  ..., -0.6503,  1.5496,  0.1571],
         [ 0.0842,  1.2824, -0.1798,  ...,  1.0897, -1.0470,  0.1097],
         ...,
         [ 0.5834, -0.4746, -0.4125,  ..., -1.2188,  0.1011,  0.3760],
         [-2.0657,  1.5787, -1.8359,  ...,  0.5942,  0.7440,  0.1324],
         [ 0.1157,  0.4007, -1.0944,  ..., -0.4286, -0.3150, -1.5131]]),
 tensor([[-0.7269, -0.0560, -1.8581,  ..., -0.1133, -1.1299,  0.1256],
         [-0.0855, -0.3944,  0.4496,  ..., -0.4466, -0.8667, -2.1378],
         [ 0.8677, -0.0500,  0.0177,  ..., -0.6424, -0.9377, -1.6000],
         ...,
         [-0.8026, -0.3866,  0.0642,  ..., -0.8097,  0.7335, -1.0225],
         [-0.7219,  0.1388, -2.2338,  ...,  0.0934,  0.9332,  1.1136],
         [ 0.0671, -1.8092,  0.5348,  ..., -1.2397, -0.2721,  0.6525]]))

In [43]:
from time import time

In [ ]:
begin = time()
C = torch.mm(A, B)
print('time for 1.2: ', (time() - begin))

time for 1.2:  0.2332909107208252


In [ ]:
begin = time()
C = torch.mv(A, B[:, 0]).reshape(4096, 1)
for i in range(1, 4096):
    C = torch.cat((C, torch.mv(A, B[:, i]).reshape(4096, 1)), dim=1)
print(C.shape)
print('time for 1.3: ', (time() - begin))

torch.Size([4096, 4096])
time for 1.3:  27.706761121749878


In [ ]:
begin = time()
C = torch.empty((4096, 4096))
for i in range(4096):
    for j in range(4096):
        C[i, j] = torch.dot(A[i, :], B[:, j])
print('time for 1.4: ', time() - begin)

time for 1.4:  326.3792338371277


## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

> It is useful to work with $B$ since it is a square, positiive semidefinite matrix. It seems to hold all the information of $B$ but transform an arbitrary matrix into one with nice properties. When we do SVD we look at $AA^T$, so now we can specify which entries to leave out by setting some of $D$'s diagonal values to zero or which entries to modify by setting $D$'s diagonal values to not $1$.

> If say $A$ is a data matrix, it is better to work with it directly when training a machine learning algorithm.

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [3]:
!nvidia-smi

Thu Dec 16 08:37:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
X = torch.rand([2, 2], device=torch.device('cuda'))
X

tensor([[0.6671, 0.6956],
        [0.7073, 0.3004]], device='cuda:0')

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [39]:
import numpy as np

In [41]:
m = torch.distributions.normal.Normal(torch.zeros((4096, 4096)), torch.tensor([1.]))
A = m.sample()
B = m.sample()

In [48]:
begin = time()
c = np.zeros(4096)
for i in range(4096):
    c[i] = torch.mv(A, B[:, i]).norm() ** 2
print('time for 4.1: ', time() - begin)

time for 4.1:  8.573265790939331


In [51]:
begin = time()
d = torch.zeros(4096, dtype=torch.float32)
for i in range(4096):
    d[i] = (torch.mv(A, B[:, i]).norm() ** 2)
c = d.numpy()
print('time for 4.2: ', time()- begin)

time for 4.2:  8.88628602027893


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [52]:
C = torch.empty(A.shape)
C += torch.mm(A, B)

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [60]:
x = torch.arange(-10, 10, 0.1).reshape((200, 1))
A = x ** torch.arange(1, 21).reshape((1, 20))
A

tensor([[-1.0000e+01,  1.0000e+02, -1.0000e+03,  ...,  1.0000e+18,
         -1.0000e+19,  1.0000e+20],
        [-9.9000e+00,  9.8010e+01, -9.7030e+02,  ...,  8.3451e+17,
         -8.2617e+18,  8.1791e+19],
        [-9.8000e+00,  9.6040e+01, -9.4119e+02,  ...,  6.9514e+17,
         -6.8123e+18,  6.6761e+19],
        ...,
        [ 9.7000e+00,  9.4090e+01,  9.1267e+02,  ...,  5.7795e+17,
          5.6061e+18,  5.4379e+19],
        [ 9.8000e+00,  9.6040e+01,  9.4119e+02,  ...,  6.9514e+17,
          6.8123e+18,  6.6761e+19],
        [ 9.9000e+00,  9.8010e+01,  9.7030e+02,  ...,  8.3451e+17,
          8.2617e+18,  8.1791e+19]])